# Object Detection

[Docker Containers](https://github.com/ctuning/ck-object-detection/tree/master/docker)

Neural Network models include:
* [SSD-MobileNet]
* [SSD-ResNet50]
* [SSD-FPN]
* [Faster-RCNN Inception ResNet v2]
* [Faster-RCNN NAS]
* [Faster-RCNN NAS lowproposals]
* [Faster-RCNN ResNet101 lowproposals]
* [Faster-RCNN ResNet50 lowproposals]

## Table of Contents

1. [Overview](#overview)
1. [Platform](#platform)
1. [Experimental data](#data) [for developers]
1. [Data wrangling code](#code) [for developers]
1. **TODO**

<a id="overview"></a>
## Overview

This Jupyter Notebook studies performance (execution time) vs accuracy (mAP and Recall) of different Object Detection networks, on different size objects (large, medium and small).
Moreover the experiments are performed on different architectures, in particular CPU and GPU, to evaluate the benefit of GPU acceleration.

<a id="platform"></a>
## Platform

<a id="CPU info"></a>
### CPU

  - Model:
    - Intel Xeon
  - Version:
    - E5-2650 v3;  
  - Frequency:
    - 2.30GHz;
  - Number of Cores (physical):
    - 10
  - HyperThreading
    - Yes


  - RAM:
    - 32 GB;

  - BSP:
    - Ubuntu 16.04 LTS Linux


<a id="GPU info"></a>
### GPU

  - Model:
    - NVIDIA GeForce GTX 1080
  - Frequency:
    - 1.6GHz
  - RAM:
    - 8 GB  
  - CUDA Version:
    - 10.2
  - Driver Version:
    - 430.14
  - MEMORY:
    - 8 GB;


<a id="data"></a>
## Get the experimental data

<a id="code"></a>
## Data wrangling code

**NB:** Please ignore this section if you are not interested in re-running or modifying this notebook.

### Includes

#### Standard

In [ ]:
import os
import sys
import json
import re

#### Scientific

If some of the scientific packages are missing, please install them using:
```
# pip install jupyter pandas numpy matplotlib
```

In [ ]:
import IPython as ip
import pandas as pd
import numpy as np
import matplotlib as mp
import seaborn as sb

In [ ]:
print ('IPython version: %s' % ip.__version__)
print ('Pandas version: %s' % pd.__version__)
print ('NumPy version: %s' % np.__version__)
print ('Matplotlib version: %s' % mp.__version__)
print ('Seaborn version: %s' % sb.__version__)

In [ ]:
from IPython.display import Image, display
def display_in_full(df):
    pd.options.display.max_columns = len(df.columns)
    pd.options.display.max_rows = len(df.index)
    display(df)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [ ]:
default_colormap = cm.autumn
default_fontsize = 16
default_barwidth = 0.8
default_figwidth = 24
default_figheight = 3
default_figdpi = 200
default_figsize = [default_figwidth, default_figheight]

In [ ]:
if mp.__version__[0]=='2': mp.style.use('classic')
mp.rcParams['figure.max_open_warning'] = 200
mp.rcParams['figure.dpi'] = default_figdpi
mp.rcParams['font.size'] = default_fontsize
mp.rcParams['legend.fontsize'] = 'medium'

In [ ]:
save_fig_ext = 'png'
save_fig_dir = os.path.join(os.path.expanduser("~"), 'omnibenchmark')
if not os.path.exists(save_fig_dir):
    os.mkdir(save_fig_dir)

#### Collective Knowledge

If CK is not installed, please install it using:
```
# python -m pip install ck
```

In [ ]:
import ck.kernel as ck
print ('CK version: %s' % ck.__version__)

#### Experimental data

Download experimental data and add CK repositories as follows:
```
$ wget https://www.dropbox.com/s/0mxkvlstico349n/ckr-medium-object-detection-accuracy.zip
$ ck add repo --zip=ckr-medium-object-detection-accuracy.zip

$ wget https://www.dropbox.com/s/zy68dsmp1yzv703/ckr-medium-object-detection-performance-docker.zip
$ ck add repo --zip=ckr-medium-object-detection-performance-docker.zip

$ wget https://www.dropbox.com/s/7829e4zmmbgkqyu/ckr-medium-object-detection-performance-native.zip
$ ck add repo --zip=ckr-medium-object-detection-performance-native.zip
```

In [ ]:
repo_uoa = 'medium-object-detection-accuracy'
!ck list $repo_uoa:experiment:* --print_full | sort
print ("")
print ("*"*80)
print ("")
perf_docker_repo_uoa = 'medium-object-detection-performance-docker'
!ck list $perf_docker_repo_uoa:experiment:* --print_full | sort
print ("")
print ("*"*80)
print ("")
perf_native_repo_uoa = 'medium-object-detection-performance-native'
!ck list $perf_native_repo_uoa:experiment:* --print_full | sort

### Access experimental data

In [ ]:
def get_experimental_results(repo_uoa, tags='', accuracy=True,
                             module_uoa='experiment', _library=None, _platform=None):
    r = ck.access({'action':'search', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'tags':tags})
    #from pprint import pprint
    #pprint (r)
    if r['return']>0:
        print('Error: %s' % r['error'])
        exit(1)
    experiments = r['lst']

    dfs = []
    for experiment in experiments:
        data_uoa = experiment['data_uoa']
        r = ck.access({'action':'list_points', 'repo_uoa':repo_uoa, 'module_uoa':module_uoa, 'data_uoa':data_uoa})
     
        pipeline_file_path = os.path.join(r['path'], 'pipeline.json')
        with open(pipeline_file_path) as pipeline_file:
            pipeline_data_raw = json.load(pipeline_file)
        weights_env = pipeline_data_raw['dependencies']['weights']['dict']['env']
        tags = r['dict']['tags']

        print (tags)
        for point in r['points']:
            point_file_path = os.path.join(r['path'], 'ckp-%s.0001.json' % point)
            with open(point_file_path) as point_file:
                point_data_raw = json.load(point_file)
            characteristics_list = point_data_raw['characteristics_list']

            #pprint (point_data_raw['choices']['env'])
            num_repetitions = len(characteristics_list)
            platform = point_data_raw['features']['platform']['platform']['model']
            #if _platform and _platform!=platform: continue
            img_width = np.int64(point_data_raw['choices']['env'].get('CK_ENV_IMAGE_WIDTH',-1))
            img_height = np.int64(point_data_raw['choices']['env'].get('CK_ENV_IMAGE_HEIGHT',-1))
            if np.int64(point_data_raw['choices']['env'].get('CK_ENABLE_BATCH',-1))==1:
                batch_en = True 
                batch_size = np.int64(point_data_raw['choices']['env'].get('CK_BATCH_SIZE',-1))
                batch_count = np.int64(point_data_raw['choices']['env'].get('CK_BATCH_COUNT',-1))
            else :
                batch_size = 1
                batch_en = False 
                batch_count = np.int64(point_data_raw['choices']['env'].get('CK_BATCH_SIZE',-1))*np.int64(point_data_raw['choices']['env'].get('CK_BATCH_COUNT',-1))

            characteristics = characteristics_list[0]
            if accuracy:
                data = [
                    {
                        'model': tags[0],
                        'tf_version':'cuda',
                        'batch_size': batch_size,
                        'batch_count': batch_count,
                        'batch_en': batch_en,
                        'img_height': img_height,
                        'img_width': img_width,
                        'num_reps':1,
                        
                        # runtime characteristics
                        'Recall':     characteristics['run'].get('recall',0)*100,
                        'mAP':        characteristics['run'].get('mAP',0)*100,
                        'mAP_large':  characteristics['run']['metrics'].get('DetectionBoxes_Recall/AR@100 (large)', 0)*100,
                        'mAP_medium': characteristics['run']['metrics'].get('DetectionBoxes_Recall/AR@100 (medium)', 0)*100,
                        'mAP_small':  characteristics['run']['metrics'].get('DetectionBoxes_Recall/AR@100 (small)', 0)*100,
                    }
                ]
                print(data[0]['model'])
            else: # performance
                ####### this conversion is still needed because some of the result have the old naming convention
                tf_version = 'default'
                trt = point_data_raw['choices']['env'].get('CK_ENABLE_TENSORRT',0) 
                trt_dyn = point_data_raw['choices']['env'].get('CK_TENSORRT_DYNAMIC',0)
                
                print (point_data_raw['choices']['env'].get('CK_ENABLE_TENSORRT',0),trt)        
                print (point_data_raw['choices']['env'].get('CK_TENSORRT_DYNAMIC',0),trt_dyn)        
                if trt_dyn == '1':
                    tf_version = 'tensorrt-dynamic'
                elif trt == '1':
                    tf_version = 'tensorrt'
                elif tags[0] == 'tensorrt' or tags[0] =='source-cuda':
                    tf_version = 'cuda'
                elif tags[0] == 'tf-src-cpu' or tags[0] =='source-cpu':
                    tf_version = 'cpu'
                elif tags[0] == 'tf-prebuild-cpu' or tags[0] == 'prebuilt-cpu':
                    tf_version = 'cpu-prebuilt'
                else:
                    tf_version = tags[0]

                model = tags[1]
                data = [
                    {
                        'model': model,
                        'tf_version':tf_version,
                        'batch_size': batch_size,
                        'batch_count': batch_count,
                        'batch_en': batch_en,
                        'img_height': img_height,
                        'img_width':img_width,
                        'num_reps' : num_repetitions,
                        # statistical repetition
                        'repetition_id': repetition_id,
                        # runtime characteristics
                        'avg_fps': characteristics['run'].get('avg_fps', 'n/a')*batch_size,
                        'avg_time_ms': characteristics['run']['avg_time_ms']/batch_size,
                        'graph_load_time_ms': characteristics['run']['graph_load_time_s']*1e+3,
                        'images_load_time_avg_ms': characteristics['run']['images_load_time_avg_s']*1e+3,
                    }
                    for (repetition_id, characteristics) in zip(range(num_repetitions), characteristics_list)
                ]
                print(data[0]['tf_version'])
            index = [
                'model', 'tf_version', 'batch_size', 'batch_count','batch_en','img_height','img_width','num_reps'
            ]
            # Construct a DataFrame.
            df = pd.DataFrame(data)
            df = df.set_index(index)
            # Append to the list of similarly constructed DataFrames.
            dfs.append(df)
    if dfs:
        # Concatenate all thus constructed DataFrames (i.e. stack on top of each other).
        result = pd.concat(dfs)
        result.sort_index(ascending=True, inplace=True)
    else:
        # Construct a dummy DataFrame the success status of which can be safely checked.
        result = pd.DataFrame(columns=['success?'])
    return result
!ck recache repo
dfs = get_experimental_results(repo_uoa)

dfs_perf = get_experimental_results(perf_docker_repo_uoa,accuracy=False)

dfs_perf_native = get_experimental_results(perf_native_repo_uoa,accuracy=False)

### Plot experimental data

In [ ]:
display_in_full(dfs)
display_in_full(dfs_perf)

#### Plot accuracy (bar plot)

In [ ]:
def plot_accuracy_compared(df_raw, groupby_level='img_height',
                           save_fig=False,save_fig_name='acc_compared',
                           performance_metric=['mAP','mAP_large','mAP_medium','mAP_small'],
                           title=None, figsize=None, rot=90):
    df_bar = pd.DataFrame(
        data=df_raw[performance_metric].values, columns=performance_metric,
        index=pd.MultiIndex.from_tuples(
            tuples=[ (l,be,s,c) for (l,_,_,_,be,s,c,_) in df_raw.index.values ],
            names=[ 'model', 'batch_en','img_height', 'img_width' ]
        )
    )
    for index, row in df_bar.iterrows():
        (model,batch_en, img_height,img_width) = index
        if model == 'yolo-v3':
            df_bar.loc[(model, True ,-1 ,-1)] = df_bar.loc[(model, False ,-1 ,-1)][performance_metric]            

    unstack_level = 'batch_en'
    import matplotlib
    aaa = ['lightcoral','cyan','indianred','darkturquoise','brown','cadetblue','darkred','steelblue']
    colormap = matplotlib.colors.ListedColormap(aaa, name='from_list', N=8)
    xlabel='model'
    xtics = df_bar.groupby(level=df_bar.index.names[:-1]).median().index.get_level_values('model').drop_duplicates()
    display_in_full(df_bar)
    ylabel='mAP %'
    if not title: title = 'Intentionally left blank' 
    if not figsize: figsize = [default_figwidth, 8]

    # Plot 
    mean = df_bar.groupby(level=df_bar.index.names[:-1]).mean().unstack(unstack_level)
    std = df_bar.groupby(level=df_bar.index.names[:-1]).std().unstack(unstack_level)
    axes = mean.groupby(level=groupby_level) \
        .plot(yerr=std, kind='bar', grid=True, width=0.8, rot=rot, figsize=figsize,
              fontsize=default_fontsize, colormap=colormap)
    for count, ax in enumerate(axes):
        # Title.
        ax.set_title('Accuracy drop from external resizing')
        # X label.
        ax.set_xlabel(xlabel)
        ax.set_xticklabels(xtics)
        # Y axis.
        ax.set_ylabel(ylabel)
        patches, labels = ax.get_legend_handles_labels()

        labels = [x.strip('(') for x in labels]
        labels = [x.strip(')') for x in labels]
        ax.legend(patches, labels, loc='best',title='Metric,isResized')
        if save_fig:
            save_fig_path = os.path.join(save_fig_dir, '%s.%s' % (save_fig_name, save_fig_ext))
            ax.figure.savefig(save_fig_path, dpi=default_figdpi, bbox_inches='tight')        

plot_accuracy_compared(dfs , rot=90,save_fig=True)

In [ ]:
def plot_accuracy(df_raw, groupby_level='batch_en', 
                  save_fig=False,save_fig_name='accuracy_',
                  performance_metric=['mAP','mAP_large','mAP_medium','mAP_small'],
                  title=None, figsize=None, rot=90):

    df_bar = pd.DataFrame(
        data=df_raw[performance_metric].values, columns=performance_metric,
        index=pd.MultiIndex.from_tuples(
            tuples=[ (l,be,s,c) for (l,_,_,_,be,s,c,_) in df_raw.index.values ],
            names=[ 'model', 'batch_en','img_height', 'img_width' ]
        )
    )

    unstack_level = 'img_height'
    colormap = cm.autumn
    xlabel='model'
    xtics = df_bar.groupby(level=df_bar.index.names[:-1]).median().index.get_level_values('model').drop_duplicates()
    ylabel='mAP %'
    resize_dim = ['Accuracy with internal resizing', 'Accuracy with external resizing']
    if not title: title = 'Intentionally left blank'
    if not figsize: figsize = [default_figwidth, 8]

    mean = df_bar.groupby(level=df_bar.index.names[:-1]).mean()
    std = df_bar.groupby(level=df_bar.index.names[:-1]).max() - mean 
    axes = mean.groupby(level=groupby_level) \
        .plot(yerr=std, kind='bar', grid=True, width=0.8, rot=rot, figsize=figsize,
              fontsize=default_fontsize, colormap=colormap)
    for count, ax in enumerate(axes):
        # Title.
        ax.set_title(resize_dim[count])
        # X label.
        ax.set_xlabel(xlabel)
        ax.set_xticklabels(xtics)
        # Y axis.
        ax.set_ylabel(ylabel)
        if save_fig:
            save_fig_path = os.path.join(save_fig_dir, '%s.%s' % (save_fig_name+resize_dim[count], save_fig_ext))
            ax.figure.savefig(save_fig_path, dpi=default_figdpi, bbox_inches='tight')        

plot_accuracy(dfs, rot=90, save_fig=True)

#### Plot performance compared (bar plot)

In [ ]:
def compare_native_perf(df_docker, df_native,
                        save_fig=False, save_fig_name='perf_docker_native',
                        title=None, figsize=None, rot=90):

    df_docker =df_docker[df_docker.index.get_level_values("batch_en").isin([True])]
  #  display_in_full(df_docker)
    df_bar = pd.merge(df_docker,df_native,how='inner',suffixes=('_docker','_native'),on=[ 'model','tf_version', 'batch_size','img_width', 'img_height' ])
    df_bar = df_bar[['avg_fps_docker','avg_fps_native']]
    df_bar['avg_fps_norm'] = df_bar['avg_fps_docker']/df_bar['avg_fps_native']
    df_bar = df_bar[['avg_fps_norm']]
   # display_in_full(df_bar)
    xtics = df_bar.groupby(level=df_bar.index.names[:-1]).median().index.get_level_values('model').drop_duplicates()
    
    unstack_level = ['batch_size','tf_version']
    colormap = cm.rainbow
    xlabel='model'
    groupby_level='img_width'
    ylabel=''
    if not title: title = df_bar.index.names[0]
    if not figsize: figsize = [default_figwidth, 8]
    import matplotlib
    
    aaa = ['red']*6 + ['yellow']*6 + ['green']*6 + ['cyan']*6 + ['blue']*6
    bla = matplotlib.colors.ListedColormap(aaa, name='from_list', N=30)
    
    reds = ['red','indianred','brown','firebrick','maroon', 'darkred']
    yellows = ['cornsilk','lemonchiffon','palegoldenrod', 'khaki', 'yellow','gold']
    greens = ['palegreen','lightgreen', 'limegreen','green','forestgreen', 'darkgreen']
    blues = ['cornflowerblue', 'royalblue', 'mediumblue','blue','navy','midnightblue']
    purples = ['orchid','fuchsia','mediumorchid','darkviolet','purple','indigo']
    aaa = reds + yellows +greens +blues +purples
    bla = matplotlib.colors.ListedColormap(aaa, name='from_list', N=30)
    # Plot 
    mean = df_bar.groupby(level=df_bar.index.names[:-1]).mean().unstack(unstack_level[1]).unstack(unstack_level[0])
    std = df_bar.groupby(level=df_bar.index.names[:-1]).std().unstack(unstack_level[1]).unstack(unstack_level[0])
    axes = mean.groupby(level=groupby_level).plot(yerr=std, kind='bar', grid=True, width=0.8, rot=rot, figsize=figsize,
              fontsize=default_fontsize,  colormap=bla)
    for num, ax in enumerate(axes):
        # Title.
        ax.set_title('Normalized FPS Comparison between Docker and Native Performance on five models from the Pareto Set')
        # X label.
        ax.set_xlabel(xlabel)
        ax.set_xticklabels(xtics)
        # Y axis.
        ax.set_ylabel(ylabel)
            # Shrink current axis by 20%
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
        patches, labels = ax.get_legend_handles_labels()

        labels = [x.strip('(avg_fps_norm,') for x in labels]
        labels = [x.strip(')') for x in labels]
        ax.legend(patches, labels, title='Backend, Batch_size', loc='center left', bbox_to_anchor=(1, 0.5),fontsize=default_fontsize)
        # Put a legend to the right of the current axis
        if save_fig:
            save_fig_path = os.path.join(save_fig_dir, '%s.%s' % (save_fig_name, save_fig_ext))
            ax.figure.savefig(save_fig_path, dpi=default_figdpi, bbox_inches='tight')        
    
    
    
    
compare_native_perf(dfs_perf ,dfs_perf_native,rot=45,save_fig=True)

In [ ]:
def plot_performance_per_backend(df_raw, groupby_level='tf_version',
                               performance_metric=['avg_fps','avg_time_ms','graph_load_time_ms','images_load_time_avg_ms'], 
                               save_fig=False, save_fig_name='perf_',
                               title=None, figsize=None, rot=0):
    from pprint import pprint
    df_bar = pd.DataFrame(
        data=df_raw[performance_metric].values, columns=performance_metric,
        index=pd.MultiIndex.from_tuples(
            tuples=[ (l,t,bs,s,ss) for (l,t,bs,_,_,s,ss,_) in df_raw.index.values ],
            names=[ 'model','tf_version','batch_size' ,'img_width','dummy' ]            
        )
    )

    unstack_level = ['batch_size']
    colormap = cm.autumn
    xlabel='model'
    ylabel='Images Per Second'
    if not title: title = df_bar.index.names[0]
    xtics = df_bar.groupby(level=df_bar.index.names[:-1]).median().index.get_level_values('model').drop_duplicates()
    
    if not figsize: figsize = [default_figwidth, 8]

    # Plot 
    mean = df_bar.groupby(level=df_bar.index.names[:-1]).mean().unstack(unstack_level)
    std = df_bar.groupby(level=df_bar.index.names[:-1]).std().unstack(unstack_level)
    axes = mean.groupby(level=groupby_level) \
        .plot(yerr=std, kind='bar', grid=True, width=0.8, rot=rot, figsize=figsize,
              fontsize=default_fontsize, colormap=colormap,legend=False)
    
    for num, ax in enumerate(axes):
        # Title.
        ax.set_title('TensorFlow with {} Backend'.format(axes.keys().get_values().item(num)))
        # X label.
        ax.set_xlabel(xlabel)
        ax.set_xticklabels(xtics)
        # Y axis.
        ax.set_ylabel(ylabel)
        patches, labels = ax.get_legend_handles_labels()
        labels = [label[1] for label in [x.split(',') for x in labels]]
        labels = [x.strip(')') for x in labels]
        ax.legend(patches, labels, loc='best',title='Batch Size')
                # Save figure.
        if save_fig:
            save_fig_path = os.path.join(save_fig_dir, '%s.%s' % (save_fig_name+axes.keys().get_values().item(num), save_fig_ext))
            ax.figure.savefig(save_fig_path, dpi=default_figdpi, bbox_inches='tight')

        
plot_performance_per_backend(dfs_perf, performance_metric=['avg_fps'],save_fig=True,rot=90)

In [ ]:
def plot_performance_per_model(df_raw, groupby_level='model',
                               performance_metric=['avg_fps','avg_time_ms','graph_load_time_ms','images_load_time_avg_ms'],
                               save_fig=False, save_fig_name='perf_',
                               title=None, figsize=None, rot=0):
    from pprint import pprint
    df_bar = pd.DataFrame(
        data=df_raw[performance_metric].values, columns=performance_metric,
        index=pd.MultiIndex.from_tuples(
            tuples=[ (l,t,bs,s,ss) for (l,t,bs,_,_,s,ss,_) in df_raw.index.values ],
            names=[ 'model','tf_version','batch_size' ,'img_width','dummy' ]            
        )
    )

    unstack_level = ['batch_size']
    colormap = cm.autumn
    xlabel='Backend'
    ylabel='Images Per Second'
    if not title: title = df_bar.index.names[0]
    xtics = df_bar.groupby(level=df_bar.index.names[:-1]).median().index.get_level_values('tf_version').drop_duplicates()
    
    if not figsize: figsize = [default_figwidth, 8]

    # Plot 
    mean = df_bar.groupby(level=df_bar.index.names[:-1]).mean().unstack(unstack_level)
    std = df_bar.groupby(level=df_bar.index.names[:-1]).std().unstack(unstack_level)
    axes = mean.groupby(level=groupby_level) \
        .plot(yerr=std, kind='bar', grid=True, width=0.8, rot=rot, figsize=figsize,
              fontsize=default_fontsize, colormap=colormap,legend=False)
    
    for num, ax in enumerate(axes):
        # Title.
        ax.set_title(axes.keys().get_values().item(num))
        # X label.
        ax.set_xlabel(xlabel)
        ax.set_xticklabels(xtics)
        # Y axis.
        ax.set_ylabel(ylabel)
        patches, labels = ax.get_legend_handles_labels()
        labels = [label[1] for label in [x.split(',') for x in labels]]
        labels = [x.strip(')') for x in labels]
        ax.legend(patches, labels, loc='best',title='Batch Size')
                # Save figure.
        if save_fig:
            save_fig_path = os.path.join(save_fig_dir, '%s.%s' % (save_fig_name+axes.keys().get_values().item(num), save_fig_ext))
            ax.figure.savefig(save_fig_path, dpi=default_figdpi, bbox_inches='tight')

        
plot_performance_per_model(dfs_perf, performance_metric=['avg_fps'],save_fig=True)

In [ ]:
def plot_performance(df_raw, groupby_level='img_width',
                     performance_metric=['avg_fps','avg_time_ms','graph_load_time_ms','images_load_time_avg_ms'], 
                     save_fig=False, save_fig_name='perf_cuda_trtdyn',
                     title=None, figsize=None, rot=0):
    from pprint import pprint
    df_bar = pd.DataFrame(
        data=df_raw[performance_metric].values, columns=performance_metric,
        index=pd.MultiIndex.from_tuples(
            tuples=[ (l,t,bs,s,ss) for (l,t,bs,_,_,s,ss,_) in df_raw.index.values ],
            names=[ 'model','tf_version','batch_size','img_width','dummy' ]
        )
    )
    items = ['cuda','tensorrt-dynamic']
    df_bar = df_bar.query('tf_version in @items')
    unstack_level = ['tf_version','batch_size']
    import matplotlib
    bla = matplotlib.colors.ListedColormap(['darkred','steelblue'], name='from_list', N=12)
    
    xlabel='Model'
    ylabel='Images Per Second'
    if not title: title = df_bar.index.names[0]
    xtics = df_bar.groupby(level=df_bar.index.names[:-1]).median().index.get_level_values('model').drop_duplicates()
    if not figsize: figsize = [default_figwidth, 8]

    # Plot.
    mean = df_bar.groupby(level=df_bar.index.names[:-1]).mean().unstack(unstack_level[1]).unstack(unstack_level[0])
    std = df_bar.groupby(level=df_bar.index.names[:-1]).std().unstack(unstack_level[1]).unstack(unstack_level[0])
    axes = mean.groupby(level=groupby_level) \
        .plot(yerr=std, kind='bar', grid=True, width=0.8, rot=rot, figsize=figsize,
              fontsize=default_fontsize, colormap=bla,legend=False)
    for num, ax in enumerate(axes):
        # Title.
        ax.set_title('CUDA Backend vs TensorRT-Dynamic Backend')
        # X label.
        ax.set_xlabel(xlabel)
        ax.set_xticklabels(xtics)
        # Y axis.
        ax.set_ylabel(ylabel)
        patches, labels = ax.get_legend_handles_labels()
        labels = [label[2] for label in [x.split(',') for x in labels]]
        labels = [x.strip(')') for x in labels]
        ax.legend(patches[:2], labels[:2], loc='best', title='Backend')

        # Save figure.
        if save_fig:
            save_fig_path = os.path.join(save_fig_dir, '%s.%s' % (save_fig_name+str(num), save_fig_ext))
            ax.figure.savefig(save_fig_path, dpi=default_figdpi, bbox_inches='tight')


plot_performance(dfs_perf, performance_metric=['avg_fps'],rot=90,save_fig=True)

In [ ]:
def merge_performance_accuracy(df_performance, df_accuracy, 
                               reference_platform=None, reference_lib=None, 
                               performance_metric='avg_fps', accuracy_metric='mAP', ideal=False):
    df = df_performance[[performance_metric]]
    accuracy_list = []
    for index, row in df.iterrows():
        (model,tf_version, batch_size, batch_count,batch_en, img_height,img_width,num_reps) = index
        if ideal:
            accuracy = df_accuracy.loc[(model,'cuda', 1, 5000, False ,-1 ,-1, 1)][accuracy_metric]            
        else:
            img_size = 'no-resize' if batch_size == 1 else 'model-resize'


            if img_size == 'no-resize' or model == 'yolo-v3':
                accuracy = df_accuracy.loc[(model,'cuda', 1, 5000, False, -1 ,-1, 1)][accuracy_metric]
            else:
                accuracy = df_accuracy.loc[(model,'cuda', 1, 5000, True,  -1 ,-1, 1)][accuracy_metric]
        accuracy_list.append(accuracy)

    #assign to the value of accuracy_metric
    kwargs = {accuracy_metric : accuracy_list}
    df = df.assign(**kwargs) 
    #display_in_full(df)
    return df

In [ ]:
#plot_utils
model_to_color = { 
        'ssd-mobilenet-v1-fpn'                                : 'red',
        'rcnn-inception-resnet-v2-lowproposals'    : 'yellow',
        'rcnn-nas-lowproposals'                   : 'orange',
        'rcnn-resnet101-lowproposals'             : 'green',
        'rcnn-inception-v2'                       : 'purple',
        'rcnn-nas-non-lowproposal'                : 'cyan',
        'ssd-inception-v2'                        : 'blue',
        'ssd-mobilenet-v1-non-quantized-mlperf'   : 'gray',
        'ssd-mobilenet-v1-quantized-mlperf'       : 'indigo',
        'ssd-resnet50-fpn'                        : 'saddlebrown',
        'ssdlite-mobilenet-v2'                    : 'teal',
        'rcnn-resnet50-lowproposals'              : 'darkgoldenrod',
        'yolo-v3'                                   : 'brown'
}

tf_to_marker = {
        'tensorrt'             : '1',
        'tensorrt-dynamic'     : '2',
        'cuda'          : '3',
        'cpu'           : '4',
        'cpu-prebuilt'      : '+'
}

resize_to_marker = {
        'no-resize'             : 'x',
        'model-resize'          : '*'

}

bs_to_size = {
        1 : 1,
        2 : 1.5,
        4 : 2,
        8 : 2.5,
        16 : 3,
        32 : 3.5
}
### not used anymore, left in case should change the policy
#     model_to_real_name = { 
#         'ssd-mobilenet-v1-fpn'                                : 'ssd_mobilenet_v1_fpn_coco',
#         'rcnn-inception-resnet-v2-lowproposals'    : 'faster_rcnn_inception_resnet_v2_atrous_lowproposals_coco',
#         'rcnn-nas-lowproposals'                   : 'faster_rcnn_nas_lowproposals_coco',
#         'rcnn-resnet101-lowproposals'             : 'faster_rcnn_resnet101_lowproposals_coco',
#         'rcnn-inception-v2'                       : 'faster_rcnn_inception_resnet_v2_atrous_coco',
#         'rcnn-nas-non-lowproposal'               : 'faster_rcnn_nas',
#         'ssd-inception-v2'                        : 'ssd_inception_v2_coco',
#         'ssd-mobilenet-v1-non-quantized-mlperf'            : 'ssd_mobilenet_v1_coco',
#         'ssd-mobilenet-v1-quantized-mlperf'                : 'ssd_mobilenet_v1_quantized_coco',
#         'ssd-resnet50-fpn'                           : 'ssd_resnet_50_fpn_coco',
#         'ssdlite-mobilenet-v2'                                : 'ssdlite_mobilenet_v2_coco',
#         'rcnn-resnet50-lowproposals'              : 'faster_rcnn_resnet50_lowproposals_coco',
#         'yolo-v3'                                   : 'yolo v3'
#     }
    
    
#         tf_to_marker = {
#         'tensorrt'             : 'x',
#         'tensorrt-dynamic'     : '*',
#         'cuda_sources'         : 'D',
#         'cuda'          : 'D',
#         'cpu'           : '<',
#         'cpu-prebuilt'      : 'o'
#     }
           
    
    

import matplotlib.lines as mlines

mark1 = mlines.Line2D([], [], color='black', marker='1', linestyle='None',
                          markersize=5, label='tensorrt')
mark2 = mlines.Line2D([], [], color='black', marker='2', linestyle='None',
                          markersize=5, label='tensorrt-dynamic')
mark3 = mlines.Line2D([], [], color='black', marker='3', linestyle='None',
                          markersize=5, label='cuda')
mark4 = mlines.Line2D([], [], color='black', marker='4', linestyle='None',
                          markersize=5, label=' cpu')
mark5 = mlines.Line2D([], [], color='black', marker='+', linestyle='None',
                          markersize=5, label=' cpu-prebuilt')
    

#    mark1 = mlines.Line2D([], [], color='black', marker='x', linestyle='None',
#                          markersize=5, label='no resize')
#    mark2 = mlines.Line2D([], [], color='black', marker='*', linestyle='None',
#                          markersize=5, label='model resize')
    
handles2 = [     mark1,mark2,mark3,mark4,mark5   ]
#handles2 = [     mark1,mark2    ]



def finalize_plot(ax,xmin, xmax, xstep, ymin, ymax, ystep,save_fig, save_fig_name,accuracy_metric):
# X axis.
    xlabel='Images Per Second'
    ax.set_xlabel(xlabel)
    ax.set_xlim(xmin, xmax)
    ax.set_xticks(np.arange(xmin, xmax, xstep))
    for xtick in ax.xaxis.get_major_ticks(): xtick.label.set_fontsize(5)
    # Y axis.
    ylabel=accuracy_metric+' %'
    ax.set_ylabel(ylabel)
    ax.set_ylim(ymin, ymax)
    ax.set_yticks(np.arange(ymin, ymax, ystep))
    for ytick in ax.yaxis.get_major_ticks(): ytick.label.set_fontsize(5)
    # Legend.
    handles = [ 
        mp.patches.Patch(color=color, label=model)
        for (model, color) in sorted(model_to_color.items())
    ]
    

    handles+=handles2
    ## in case want to move the legend outside the plot, decomment the following THREE rows (also the commented part besid plt.legend)
    #box = ax.get_position()
    #ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    plt.legend(title='', handles=handles[::-1], loc='best', prop={'size': 5})#,bbox_to_anchor=(1, 0.5),fontsize=default_fontsize)
    
    # Show with grid on.
    plt.grid(True)
    fig1 = plt.gcf()
    plt.show()
    plt.draw()
    
    # Save figure.
    if save_fig:
        save_fig_path = os.path.join(save_fig_dir, '%s.%s' % (save_fig_name, save_fig_ext))
        fig1.savefig(save_fig_path, dpi=default_figdpi, bbox_inches='tight')    



In [ ]:
def plot(ideal=False, performance_metric='avg_fps', accuracy_metric='mAP',
         xmin=0.0, xmax=85.01, xstep=5, ymin=22, ymax=46.01, ystep=4,
         title=None, save_fig=False, save_fig_name='full_dse'):
    fig = plt.figure(figsize=(8,4), dpi=default_figdpi)
    ax = fig.gca()
    if ideal:
        save_fig_name=save_fig_name+'ideal'
        ax.set_title('Full Space Exploration with Ideal Accuracy')
        df_performance_accuracy=merge_performance_accuracy(dfs_per, dfs,performance_metric="avg_fps", accuracy_metric=accuracy_metric,ideal=True )
    else :
        ax.set_title('Full Space Exploration with Measured Accuracy')
        df_performance_accuracy=merge_performance_accuracy(dfs_per, dfs,performance_metric="avg_fps", accuracy_metric=accuracy_metric,ideal=False )

    df = df_performance_accuracy
    df = df.groupby(level=df.index.names[:-1]).mean()
    #display_in_full(df)
    for index, row in df.iterrows():
        (model,tf_version, batch_size, batch_count,batch_en, img_height,img_width) = index
        performance = row[performance_metric]
        accuracy = row[accuracy_metric]
        
        # Mark Pareto-optimal points.
        is_on_pareto = True
        for index1, row1 in df.iterrows():
            is_no_slower = row1[performance_metric] >= row[performance_metric]
            is_no_less_accurate = row1[accuracy_metric] >= row[accuracy_metric]
            is_faster = row1[performance_metric] > row[performance_metric]
            is_more_accurate = row1[accuracy_metric] > row[accuracy_metric]
            if ((is_faster and is_no_less_accurate) or (is_more_accurate and is_no_slower)):

                is_on_pareto = False
                break

        # Select size, color and marker.
        size = bs_to_size[batch_size]*4+2 #resolution / 16
        color = model_to_color[model]
        marker = tf_to_marker[tf_version]#resize_to_marker[img_width]#

        # Plot.
        ax.plot(performance, accuracy, marker, markerfacecolor=color, markersize=size,markeredgecolor=color)

        # Mark Pareto-optimal points with scaled black pluses.
        if is_on_pareto:
            ax.plot(performance, accuracy, 'o', markersize=4,markerfacecolor='black',markeredgecolor='black')

    finalize_plot(ax,xmin, xmax, xstep, ymin, ymax, ystep,save_fig, save_fig_name,accuracy_metric)

        
plot (ideal=True,accuracy_metric='mAP_small',ymin=0, ymax=25.01,save_fig=True,save_fig_name='small_full_dse_')
plot (ideal=False,accuracy_metric='mAP_large',ymin=30, ymax=80.01,save_fig=True,save_fig_name='large_full_dse_')
plot (ideal=False,save_fig=True)

In [ ]:
def plot_max(ideal=False, performance_metric='avg_fps', accuracy_metric='mAP',
         xmin=0.0, xmax=85.01, xstep=5, ymin=22, ymax=46.01, ystep=4,
         title=None, save_fig=False, save_fig_name='best_speed'):
    fig = plt.figure(figsize=(8,4), dpi=default_figdpi)
    ax = fig.gca()
    if ideal:
        ax.set_title('Faster Configuration with Ideal Accuracy')
        save_fig_name=save_fig_name+'ideal'
        df_performance_accuracy=merge_performance_accuracy(dfs_per, dfs,performance_metric="avg_fps", accuracy_metric=accuracy_metric,ideal=True )
    else :
        ax.set_title('Fastest Configuration with Measured Accuracy')
        df_performance_accuracy=merge_performance_accuracy(dfs_per, dfs,performance_metric="avg_fps", accuracy_metric=accuracy_metric,ideal=False )

    df = df_performance_accuracy
    df = df.groupby(level=df.index.names[:-1]).mean()
    df = df.groupby(level=df.index.names[:-4]).max()
    display_in_full(df)
    points_to_plot=[]
    for index, row in df.iterrows():
        (model,tf_version, batch_size) = index
        performance = row[performance_metric]
        accuracy = row[accuracy_metric]
        plot = True
        # Analyze point of same model
        is_on_pareto = True
        for index1, row1 in df.iterrows():
            if index == index1:
                continue
            if index1[0] != model:
                continue
            is_faster = row1[performance_metric] > row[performance_metric]
            if is_faster:
                plot = False
                continue
        
        
        if plot:
        # Select size, color and marker.
        #no faster points have been found with the same model.
            size = bs_to_size[batch_size]*4+2 #resolution / 16
            color = model_to_color[model]
            marker = tf_to_marker[tf_version]#resize_to_marker[img_width]#

        # Plot.
            ax.plot(performance, accuracy, marker, markerfacecolor=color, markersize=size,markeredgecolor=color)

        # Mark Pareto-optimal points with scaled black pluses.

    finalize_plot(ax,xmin, xmax, xstep, ymin, ymax, ystep,save_fig, save_fig_name,accuracy_metric)
    
plot_max(ideal=True,save_fig=True)

#### Resizing layer in the network

There are two types of resizing: __fixed__ and __keep_aspect_ratio__.

The first one takes as input the image and returns an image of fixed dimensions, changing from network to network.
Using this layer are the following networks:

       - 'ssd_mobilenet_v1_fpn_coco', 640*640
       - 'faster_rcnn_nas_lowproposals_coco', 1200*1200
       - 'faster_rcnn_nas', 1200*1200
       - 'ssd_inception_v2_coco', 300*300
       - 'ssd_mobilenet_v1_coco', 300*300
       - 'ssd_mobilenet_v1_quantized_coco', 300*300
       - 'ssd_resnet_50_fpn_coco', 640*640
       - 'ssdlite_mobilenet_v2_coco', 300*300
       - 'yolo v3', 416*416
       
The second one behaviour is actually not completely clear, and have a minimum and maximum dimension of the output images. network using this layer are:

       - 'faster_rcnn_inception_resnet_v2_atrous_lowproposals_coco', min: 600  max: 1024
       - 'faster_rcnn_inception_resnet_v2_atrous_coco', min: 600  max: 1024
       - 'faster_rcnn_resnet101_lowproposal_coco', min: 600  max: 1024
       - 'faster_rcnn_resnet50_lowproposals_coco', min: 600  max: 1024
       
From the analysis on the performance-accuracy benchmarks, it seems that the models using fixed resizing performs better than the one with the keep aspect ratio.

yolo is actually in the middle ground: its preprocessing is doing a fixed resize to 416 * 416, however the resize is done keeping the aspect ratio and padding, outside the network. the network doesn't perform any resizing, but takes 416 * 416 images as input.